In [11]:
#Warning
import warnings
warnings.filterwarnings("ignore")

#Main
import os
import glob
import cv2

In [12]:
import  numpy   as np
import pandas as pd
import gc
import string
import time
import time
import random
import imutils
from PIL import Image
from tqdm import tqdm
tqdm.pandas()

#Visualisation
import matplotlib
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.manifold import TSNE

#Model
from sklearn.model_selection import train_test_split

In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from keras.models import load_model, Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [24]:
#Configuration
class CFG: 
    batch_size = 64
    img_height = 64
    img_width = 64
    epochs = 10
    num_classes = 29
    img_channels =3
    
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"]  = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    

In [15]:
#Labels
TRAIN_PATH = "asl_alphabet_train/asl_alphabet_train/"
labels = []
alphabet = list(string.ascii_uppercase)
labels.extend(alphabet)
labels.extend(["del","nothing","space"])
print(labels)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


## Data PreProcessing
This code creates metadata for the ASL alphabet dataset by getting all the image files for each label, creating a list of image paths, and a corresponding list of labels. It then creates a Pandas dataframe using these two lists with columns "image_path" and "label".

In [16]:
#Create Metadata

list_path = []
list_labels = []

for label in labels:
    label_path = os.path.join(TRAIN_PATH, label, "*")
    image_files = glob.glob(label_path)
    
    sign_label=[label]* len(image_files)
    
    list_path.extend(image_files)
    list_labels.extend(sign_label)
    
metadata=pd.DataFrame({
    
    "image_path":list_path,
    "label":list_labels    
})

metadata

,image_path,label
0,asl_alphabet_train/asl_alphabet_train/A/A402.jpg,A
1,asl_alphabet_train/asl_alphabet_train/A/A218.jpg,A
2,asl_alphabet_train/asl_alphabet_train/A/A1790.jpg,A
3,asl_alphabet_train/asl_alphabet_train/A/A2874.jpg,A
4,asl_alphabet_train/asl_alphabet_train/A/A2443.jpg,A
...,...,...
86995,asl_alphabet_train/asl_alphabet_train/space/sp...,space
86996,asl_alphabet_train/asl_alphabet_train/space/sp...,space
86997,asl_alphabet_train/asl_alphabet_train/space/sp...,space
86998,asl_alphabet_train/asl_alphabet_train/space/sp...,space


In [17]:
#Split Dataset to train 0.7, Val 0.15 and Test 0.15
X_train, X_test, y_train, y_test = train_test_split(
    metadata["image_path"],metadata["label"],
    test_size=0.15,
    random_state=2023,shuffle=True,
    stratify=metadata["label"]
)

data_train = pd.DataFrame({
    "image_path": X_train,
    "label": y_train
})

X_train, X_val, y_train, y_val = train_test_split(
    data_train["image_path"], data_train["label"],
    test_size=0.15/0.70,
    random_state=2023,
    shuffle=True,
    stratify=data_train["label"]
)

data_train = pd.DataFrame({
    "image_path": X_train,
    "label": y_train
})

data_val = pd.DataFrame({
    "image_path": X_val,
    "label": y_val
})

data_test = pd.DataFrame({
    "image_path": X_test,
    "label": y_test
})

display(data_train)
display(data_val)
display(data_test)

,image_path,label
51901,asl_alphabet_train/asl_alphabet_train/R/R934.jpg,R
51887,asl_alphabet_train/asl_alphabet_train/R/R1682.jpg,R
5050,asl_alphabet_train/asl_alphabet_train/B/B2380.jpg,B
29446,asl_alphabet_train/asl_alphabet_train/J/J1463.jpg,J
58405,asl_alphabet_train/asl_alphabet_train/T/T232.jpg,T
...,...,...
47111,asl_alphabet_train/asl_alphabet_train/P/P468.jpg,P
83611,asl_alphabet_train/asl_alphabet_train/nothing/...,nothing
67329,asl_alphabet_train/asl_alphabet_train/W/W2606.jpg,W
85296,asl_alphabet_train/asl_alphabet_train/space/sp...,space


,image_path,label
24961,asl_alphabet_train/asl_alphabet_train/I/I986.jpg,I
71053,asl_alphabet_train/asl_alphabet_train/X/X1169.jpg,X
73637,asl_alphabet_train/asl_alphabet_train/Y/Y1992.jpg,Y
72875,asl_alphabet_train/asl_alphabet_train/Y/Y1445.jpg,Y
40669,asl_alphabet_train/asl_alphabet_train/N/N1142.jpg,N
...,...,...
16474,asl_alphabet_train/asl_alphabet_train/F/F560.jpg,F
62741,asl_alphabet_train/asl_alphabet_train/U/U2288.jpg,U
32281,asl_alphabet_train/asl_alphabet_train/K/K535.jpg,K
20451,asl_alphabet_train/asl_alphabet_train/G/G81.jpg,G


,image_path,label
73327,asl_alphabet_train/asl_alphabet_train/Y/Y1672.jpg,Y
85482,asl_alphabet_train/asl_alphabet_train/space/sp...,space
65607,asl_alphabet_train/asl_alphabet_train/V/V547.jpg,V
53903,asl_alphabet_train/asl_alphabet_train/R/R350.jpg,R
64425,asl_alphabet_train/asl_alphabet_train/V/V2193.jpg,V
...,...,...
40578,asl_alphabet_train/asl_alphabet_train/N/N2010.jpg,N
28949,asl_alphabet_train/asl_alphabet_train/J/J198.jpg,J
42094,asl_alphabet_train/asl_alphabet_train/O/O1493.jpg,O
5425,asl_alphabet_train/asl_alphabet_train/B/B1918.jpg,B


## Data Augmentation
This code defines a function named data_augmentation() which generates image data generators using the ImageDataGenerator class from the Keras library. The function defines three generators for training, validation, and testing datasets respectively using the flow_from_dataframe() method of the ImageDataGenerator class. The generators take the image path and label information from dataframes data_train, data_val, and data_test and apply the same data augmentation technique of rescaling the image pixel values to a range of 0 to 1. The batch size, image size, and number of classes are also defined using the CFG class attributes.

In [18]:
#Data Augmentation (Just Rescale)
def data_augmentation():
    datagen = ImageDataGenerator(rescale=1/255.,)
    #Training Dataset
    train_generator = datagen.flow_from_dataframe(
        data_train,
        directory="./",
        x_col="image_path",
        y_col="label",
        class_mode="categorical",
        batch_size=CFG.batch_size,
        target_size=(CFG.img_height,CFG.img_width)      
    )
    
    #Validation Dataset
    validtion_generator = datagen.flow_from_dataframe(
        data_val,
        directory="./",
        x_col="image_path",
        y_col="label",
        class_mode="categorical",
        batch_size=CFG.batch_size,
        target_size=(CFG.img_height,CFG.img_width)  
        
    )
    
    # Validation Dataset
    validation_generator = datagen.flow_from_dataframe(
        data_val,
        directory="./",
        x_col="image_path",
        y_col="label",
        class_mode="categorical",
        batch_size=CFG.batch_size,
        target_size=(CFG.img_height, CFG.img_width),
    )
    
    # Testing Dataset
    test_generator = datagen.flow_from_dataframe(
        data_test,
        directory="./",
        x_col="image_path",
        y_col="label",
        class_mode="categorical",
        batch_size=1,
        target_size=(CFG.img_height, CFG.img_width),
        shuffle=False
    )
    
    return train_generator, validation_generator, test_generator

In [19]:
seed_everything(2023)
train_generator, validation_generator, test_generator = data_augmentation()

Found 58103 validated image filenames belonging to 29 classes.
Found 15847 validated image filenames belonging to 29 classes.
Found 15847 validated image filenames belonging to 29 classes.
Found 13050 validated image filenames belonging to 29 classes.


# Model Building

In [21]:
# Load VGG16 model and modify for ASL recognition
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(CFG.img_height, CFG.img_width, CFG.img_channels))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(29, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

display(model.summary())
display(tf.keras.utils.plot_model(model, to_file='vgg16.png', show_shapes=True))


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0   

None

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


None

## Training

In [22]:
# Compile and train the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
checkpoint = ModelCheckpoint('asl_vgg16_best_weights.h5', save_best_only=True, monitor='val_accuracy', mode='max')

In [25]:
# Train the Model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // CFG.batch_size,
    epochs=CFG.epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // CFG.batch_size,
    callbacks=[checkpoint]
)

Epoch 1/10


2023-07-01 03:31:52.124888: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


670/907 [=====================>........] - ETA: 2:19 - loss: 1.3024 - accuracy: 0.5957

KeyboardInterrupt: 

# Model Evaluation

## Model Testing


In [ ]:
scores = model.evaluate(test_generator)
print("%s: %.2f%%" % ("Evaluate Test Accuracy", scores[1]*100))

## Training Loss and Metrics


In [ ]:
# Visualize Training and Validation Results

# Create Subplot
fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=["Model Loss", "Model Accuracy"], 
)

# Configuration Plot
class PlotCFG:
    marker_size = 8
    line_size = 2
    train_color = "#76503d"
    valid_color = "#deb392"

# Loss Plot
loss = history.history['loss']
val_loss = history.history['val_loss']
fig.add_trace(
    go.Scatter(
        x=np.arange(1, len(loss)+1), y=loss,
        mode="markers+lines",
        marker=dict(
            color=PlotCFG.train_color, size=PlotCFG.marker_size,
            line=dict(color="White", width=0.5)
        ),
        line=dict(color=PlotCFG.train_color, width=PlotCFG.line_size),
        name="Training Loss"
    ), row=1, col=1
)
fig.add_trace(
    go.Scatter(
        x=np.arange(1, len(val_loss)+1), y=val_loss,
        mode="markers+lines",
        marker=dict(
            color=PlotCFG.valid_color, size=PlotCFG.marker_size,
            line=dict(color="White", width=0.5)
        ),
        line=dict(color=PlotCFG.valid_color, width=PlotCFG.line_size),
        name="Validation Loss"
    ), row=1, col=1
)

# Accuracy Plot
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
fig.add_trace(
    go.Scatter(
        x=np.arange(1, len(acc)+1), y=acc,
        mode="markers+lines",
        marker=dict(
            color=PlotCFG.train_color, size=PlotCFG.marker_size,
            line=dict(color="White", width=0.5)
        ),
        line=dict(color=PlotCFG.train_color, width=PlotCFG.line_size),
        name="Training Accuracy"
    ), row=1, col=2
)
fig.add_trace(
    go.Scatter(
        x=np.arange(1, len(val_acc)+1), y=val_acc,
        mode="markers+lines",
        marker=dict(
            color=PlotCFG.valid_color, size=PlotCFG.marker_size,
            line=dict(color="White", width=0.5)
        ),
        line=dict(color=PlotCFG.valid_color, width=PlotCFG.line_size),
        name="Validation Accuracy"
    ), row=1, col=2
)

# Update Axes
fig.update_xaxes(title="Epochs", linecolor="Black", ticks="outside", row=1, col=1)
fig.update_xaxes(title="Epochs", linecolor="Black", ticks="outside", row=1, col=2)
fig.update_yaxes(title="Categorical Loss", linecolor="Black", ticks="outside", row=1, col=1)
fig.update_yaxes(title="Accuracy", linecolor="Black", ticks="outside", row=1, col=2)

# Update Layout
fig.update_layout(
    title="Training Loss and Metrics", title_x=0.5,
    width=950, height=400,
    showlegend=False,
    plot_bgcolor="White",
    paper_bgcolor="White"
)

# Show
fig.show(iframe_connected=True)

## Confusion Matrix


In [ ]:
# Confusion Matrix
fine_tuned_model = load_model("/kaggle/working/asl_vgg16_best_weights.h5")
predictions = fine_tuned_model.predict(test_generator)

# Get the true labels from the generator
true_labels = test_generator.classes

# Compute the confusion matrix using tf.math.confusion_matrix
confusion_matrix = tf.math.confusion_matrix(
        labels=true_labels,
        predictions=predictions.argmax(axis=1),
        num_classes=29)

In [ ]:
# Create Figure
fig = go.Figure()

# Heatmap
fig.add_trace(
    go.Heatmap(
        z=confusion_matrix,
        x=labels,
        y=labels,
        text=confusion_matrix,
        texttemplate="<b>%{text}</b>",
        textfont={"size":8},
        colorscale=[[0, '#f4f4f4'],[1.0, '#76503d']],
        showscale = False,
        ygap = 5,
        xgap = 5,
        hovertemplate=
        '''
        Actual: %{y}<br>
        Predicted: %{x}<br>
        Total: %{text}
        ''',
        name="Confusion Matrix"
    )
)

# Update Axes
fig.update_xaxes(title="<b>Predicted Values</b>", tickfont_size=10)
fig.update_yaxes(title="<b>Actual Values</b>", tickfont_size=10)

# Update Layout
fig.update_layout(title_text='Confusion Matrix', title_x=0.5, font_size=14,
                  width=1050, 
                  height=1115,
                  plot_bgcolor='white',
                  showlegend=False,
)

# Show
fig.show()